### This notebook showcases how to use gradient descent helper functions


In [1]:
import json
import time
from pathlib import Path

import numpy as np
import qibo
from qibo import hamiltonians, set_backend
from boostvqe.models.dbi.double_bracket import (
    DoubleBracketGeneratorType,
    DoubleBracketIteration,
)

from boostvqe.ansatze import VQE, build_circuit
from boostvqe.utils import apply_dbi_steps, rotate_h_with_vqe
from qibo import symbols, hamiltonians
from copy import deepcopy
from boostvqe.compiling_XXZ import *

import matplotlib.pyplot as plt

## 1. Loading VQE results

In [2]:

qibo.set_backend("numpy")

# set the path string which define the results
path = "../results/vqe_data/with_params/10q7l/sgd_10q_7l_42/"

# set the target epoch to which apply DBQA and the number of steps
target_epoch = 2000
dbi_steps = 1

# upload system configuration and parameters for all the training
with open(path + "optimization_results.json") as file:
    config = json.load(file)

losses = dict(np.load(path + "energies.npz"))["0"]
params = np.load(path + f"parameters/params_ite{target_epoch}.npy")

nqubits = config["nqubits"]
# build circuit, hamiltonian and VQE
hamiltonian = hamiltonians.XXZ(nqubits=nqubits, delta=0.5)
circuit = build_circuit(nqubits, config["nlayers"], "numpy")
vqe = VQE(circuit, hamiltonian)
zero_state = hamiltonian.backend.zero_state(nqubits)
zero_state_t = np.transpose([zero_state])
target_energy = np.min(hamiltonian.eigenvalues())


# set target parameters into the VQE
vqe.circuit.set_parameters(params)
vqe_state = vqe.circuit().state()

ene1 = hamiltonian.expectation(vqe_state)


[Qibo 0.2.9|INFO|2024-06-18 10:15:40]: Using numpy backend on /CPU:0
INFO:qibo.config:Using numpy backend on /CPU:0
[Qibo 0.2.9|INFO|2024-06-18 10:15:41]: Using numpy backend on /CPU:0
INFO:qibo.config:Using numpy backend on /CPU:0


In the VQE numpy


In [3]:
print(f"VQE energy is {ene1}. \n\
The target energy is {target_energy} which means the difference is {ene1-target_energy}.\n\
The relative difference is {abs(ene1-target_energy)/abs(target_energy)}%.")

VQE energy is -14.856898957126564. 
The target energy is -15.276131122065795 which means the difference is 0.4192321649392312.
The relative difference is 0.02744360869838739%.


# 2. GCI vs GCI3 steps
$D= \sum_i B_i Z_i$
and Group commutator

In [4]:


from boostvqe.models.dbi.group_commutator_iteration_transpiler import *
from boostvqe.models.dbi.double_bracket_evolution_oracles import *


eo_xxz = XXZ_EvolutionOracle(nqubits, steps = 1, order = 2)

# implement the rotate by VQE on the level of circuits
fsoe  = VQERotatedEvolutionOracle(eo_xxz, vqe)


In [5]:
# init gci with the vqe-rotated hamiltonian
gci  = GroupCommutatorIterationWithEvolutionOracles(input_hamiltonian_evolution_oracle=fsoe, 
        mode_double_bracket_rotation=DoubleBracketRotationType.group_commutator_third_order_reduced)

#from boostvqe.models.dbi.utils_gci_optimization import *


In [102]:
def diagonal_1_pauli_z(i,n):
    """See Eq. 3 in https://arxiv.org/abs/1707.05181"""
    i += 1

    block_size = 2**(n-i)

    plus_block = [1]*block_size
    minus_block = [-1]*block_size
    block = plus_block + minus_block

    return np.array(block * (2**(i-1)))

def diagonal_product_pauli_z(i_list,n):
    """See Eq. 3 in https://arxiv.org/abs/1707.05181"""

    diagonals = []
    for i in i_list:
       diagonals.append(diagonal_1_pauli_z(i%n,n))
    return np.prod(np.array(diagonals), axis = 0)    

def dephasing_approximation(h_matrix: np.array, order = 1):
    """finds the decomposition of hamiltonian `$\Delta(h_matrix)$` into Pauli-Z operators"""
    nqubits = int(np.log2(h_matrix.shape[0]))

    diagonal = np.diag(h_matrix)

    order_1 = [ diagonal.T @ diagonal_1_pauli_z(i,nqubits) / 2**nqubits for i in range(nqubits)]
    order_2 = [ diagonal.T @ diagonal_product_pauli_z([i,j],nqubits) / 2**nqubits for i,j in itertools.product(range(nqubits),range(nqubits))]
    return {1: order_1, 2: order_2}

def nn_dephasing_approximation(h_matrix: np.array, order = 1):
    """finds the decomposition of hamiltonian `$\Delta(h_matrix)$` into Pauli-Z operators"""
    nqubits = int(np.log2(h_matrix.shape[0]))

    diagonal = np.diag(h_matrix)

    order_1 = [ diagonal.T @ diagonal_1_pauli_z(i,nqubits) / 2**nqubits for i in range(nqubits)]
    order_2 = [ diagonal.T @ diagonal_product_pauli_z([i,i+1],nqubits) / 2**nqubits for i in range(nqubits)]
    return {1: order_1, 2: order_2}

<>:22: SyntaxWarning: invalid escape sequence '\D'
<>:32: SyntaxWarning: invalid escape sequence '\D'
<>:22: SyntaxWarning: invalid escape sequence '\D'
<>:32: SyntaxWarning: invalid escape sequence '\D'
/tmp/ipykernel_40742/4203059886.py:22: SyntaxWarning: invalid escape sequence '\D'
  """finds the decomposition of hamiltonian `$\Delta(h_matrix)$` into Pauli-Z operators"""
/tmp/ipykernel_40742/4203059886.py:32: SyntaxWarning: invalid escape sequence '\D'
  """finds the decomposition of hamiltonian `$\Delta(h_matrix)$` into Pauli-Z operators"""


In [88]:
rhv = rotate_h_with_vqe(hamiltonian=hamiltonian, vqe=vqe)

In [103]:
da = nn_dephasing_approximation(rhv)
b_list2 = da[1]
j_list2 = da[2]


In [108]:
times = np.linspace(0.01,0.02,10)
eo_d = IsingNNEvolutionOracle(b_list2,j_list2,name = "D(linear)")
s,l, ls = gci.choose_step(d = eo_d,max_evals=34,step_min = 0.0081,step_max = 0.009)


In [109]:
l

-14.937939340738193